In [1]:
import numpy as np
import pandas as pd
import sys
from tqdm import tqdm
import math

In [2]:
import numba as nb

In [3]:
@nb.jit()
def nb_sum(a):
    Sum = 0
    for i in range(len(a)):
        Sum += a[i]
    return Sum

# 没用numba加速的求和函数
def py_sum(a):
    Sum = 0
    for i in range(len(a)):
        Sum += a[i]
        
    return Sum

import numpy as np
a = np.linspace(0,100,100) # 创建一个长度为100的数组

%timeit np.sum(a) # numpy自带的求和函数
%timeit sum(a) # python自带的求和函数
%timeit nb_sum(a) # numba加速的求和函数
%timeit py_sum(a)

14.8 µs ± 158 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
41.8 µs ± 571 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
The slowest run took 11.62 times longer than the fastest. This could mean that an intermediate result is being cached.
1.99 µs ± 2.68 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
58.1 µs ± 175 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [43]:
np.random.sample(10)

array([0.9456139 , 0.75524947, 0.67912952, 0.22281563, 0.75102626,
       0.03629475, 0.4868519 , 0.64976584, 0.9920782 , 0.84184192])

In [47]:
import pandas as pd
import numpy as np
df = pd.DataFrame({"data": [i for i in range(10000)]})
import random
@nb.jit
def func(df):
    for i in range(len(df)):
        df[i] = 0.1* np.random.sample(1)[0]*100
    
    return df
%timeit new_df = func(df['data'].to_numpy())
# df['data'].to_numpy()
# df

1.94 ms ± 144 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 26
model name	: Intel(R) Xeon(R) CPU           E5520  @ 2.27GHz
stepping	: 5
microcode	: 0x19
cpu MHz		: 2382.246
cache size	: 8192 KB
physical id	: 1
siblings	: 4
core id		: 0
cpu cores	: 4
apicid		: 16
initial apicid	: 16
fpu		: yes
fpu_exception	: yes
cpuid level	: 11
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx rdtscp lm constant_tsc arch_perfmon pebs bts rep_good nopl xtopology nonstop_tsc cpuid aperfmperf pni dtes64 monitor ds_cpl vmx est tm2 ssse3 cx16 xtpr pdcm dca sse4_1 sse4_2 popcnt lahf_lm pti tpr_shadow vnmi flexpriority ept vpid dtherm ida
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs itlb_multihit
bogomips	: 4521.58
clflush size	: 64
cache_alignment	: 64
address sizes	: 40 bits physical, 48 bits virtual
power management:

processor	: 1
vendor_id	: GenuineIntel
cpu family	: 6


## Important Note:
1. Default python  version is using /software/jupyterhub/env/jupyterhub-user/bin/python3.8, which is 3.8.5
2. When add conda module, the python in conda (for example: conda use 3.6 python) will overlap the default python version, then python becomes 3.6
3. Only after add conda module, Can I activate myenv
4. After activate myenv conda environment, the python 3.7.7 in myenv will overlap the python 3.6 in conda again. 

5. So Make SURE that 
    + **add conda module (python in conda) first**
    + **activate myenv (python 3.7.7 in myenv)**
    + **DO NOT add conda module again in myenv!**
    
6. **CAN Not Use pd.to_csv() to  save cleaned dataset directly**, since it converts data into string and replace most of data with "..." string. It LOSES information!
Even if we can use csv to store data without converting to string, it will take about 40G to store data again!
7. **CAN Not Use pd.to_json() to save cleaned dataset directly as well**, although it stores array correctlly without converting it to string, it takes a long time to load data as well and large space (40G) to store data as well.

8. **Final Solution:**
    + Use the API I write to load batch of data rather than full data. 
    + Randomly sample part of them as training set, test set and save it into a json file for **Quick Use**
    + For full data, **Use an API to load batch of them (recommanded for testing)** or **all of them into pandas dataframe(could take some time)**

## Strategy to Load data quicker
0. base case: use the load function I wrote to load data into array directly, but slow
1. Use the load functions i wrote to load preprocessed data directly with **MPI to load faster**
2. Save preprocessed data (data in list format, not array) to csv file. Then use literal_eval, or Spark UDF to load csv to parse **String data into list data**

In [2]:
!python --version

Python 3.8.5


## Test with my Loader Functions

In [15]:
# Testing with modules

from data_loader import load_dataset, load_PreprocessData,all_zero_means, shimmer_trended_stddev
import os
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm
import math
import random
from datetime import datetime


ModelNum = 3 #int(sys.argv[1])
EPOCHS = 30 #int(sys.argv[2])
winmin = 6 #int(sys.argv[1])
stridesec = 15 # int(sys.argv[4])

winlength = int(winmin * 60 * 15)
step = int(stridesec * 15)
start_time = datetime.now()
# data_path ='data-list.txt'
data_path ='batch-unix.txt'
meanvals, stdvals =  all_zero_means, shimmer_trended_stddev
NumFiles, data, samples_indices, labels_array = load_PreprocessData(winlength,
                                                                            step,
                                                                            meanvals, 
                                                                            stdvals,
                                                                            files_list=data_path, 
                                                                            removerest=0,
                                                                            removewalk=0,
                                                                            removebias=0)

train_set = load_dataset(data, samples_indices, labels_array)


100%|██████████| 354/354 [03:18<00:00,  1.78it/s]


Creating balanced training data array


100%|██████████| 119024/119024 [14:10<00:00, 139.93it/s]


In [32]:
train_set.data.iloc[0]

array([[-1.2206243e-02,  2.0161411e-02,  3.6075068e-01,  4.2981387e-04,
        -1.6510699e-04, -3.7603444e-04],
       [-1.2820247e-02,  2.4278400e-02,  3.5645226e-01,  4.7764552e-04,
        -5.1265582e-05, -9.0299050e-05],
       [-1.7977927e-02,  2.4803881e-02,  3.4717363e-01,  5.2298151e-04,
        -5.0981984e-05, -6.2657236e-05],
       ...,
       [ 5.5302966e-01, -1.6028321e+00, -2.9467770e-01, -4.9050752e-02,
         2.6008642e-01,  5.5196112e-01],
       [ 1.6226400e-01, -1.2037127e+00,  2.8320214e-01,  1.9215938e-01,
        -1.1539631e-01,  5.6213886e-01],
       [-3.7249854e-01, -1.5326660e+00,  7.1742004e-01, -2.5402227e-02,
        -2.6570490e-01,  3.4866059e-01]], dtype=float32)

In [33]:
train_set[train_set.label==1]

,data,label
1,"[[-0.14022358, 1.0515205, 0.34656662, 0.057980...",1
2,"[[-0.103657156, 0.03466896, -0.046756677, 0.07...",1
3,"[[-0.1583497, -0.7498793, 0.12545471, -0.02409...",1
4,"[[1.5225868, -0.39115846, 1.5109606, 0.9389003...",1
7,"[[-0.13768347, 0.16851205, 0.027480043, -0.001...",1
...,...,...
119014,"[[-0.62476885, 0.039178997, 0.26083758, -0.335...",1
119015,"[[-1.2598703, 1.0170987, -1.6936212, -0.089855...",1
119018,"[[-0.16487487, -0.5883155, -0.21024181, 0.1673...",1
119022,"[[-0.33617634, 0.0396387, 0.27841455, 0.048238...",1


In [21]:
train_set.iloc[0:5000].to_json("./data/cleaned_data_"+str(0)+".json", compression="gzip")

In [1]:
from data_loader import *

train_data, test_data= load_train_test_data(data_path ='batch-unix.txt',
                         ratio_dataset=0.05, 
                        undersampling= True,
                         test_split_ratio = 0.3,
                         winmin = 6, stridesec = 15 )

  0%|          | 0/17 [00:00<?, ?it/s]

Loading Dataset ...


 47%|████▋     | 8/17 [00:04<00:05,  1.77it/s]


KeyboardInterrupt: 

In [12]:
train_data['data'].iloc[0]

array([[-0.47285986,  0.55677134, -0.2330321 , -0.20383409,  0.00852659,
         0.2438388 ],
       [-0.6472508 ,  0.26777586, -0.07266016, -0.2460811 ,  0.07531058,
         0.28094402],
       [-0.7517754 ,  0.62782884,  0.33681288, -0.21922588,  0.04757515,
         0.2828293 ],
       ...,
       [ 0.02423186,  0.01421984,  0.49951476, -0.04769883,  0.07232166,
         0.07123255],
       [ 0.10916266, -0.11873381,  0.5735638 , -0.02974646,  0.04825988,
         0.06456374],
       [ 0.17115285, -0.14832962,  0.6142865 , -0.02937665,  0.05272228,
         0.0725497 ]], dtype=float32)

In [26]:
df = pd.read_json("data/cleaned_data_0.json",compression="gzip")

In [31]:
csv_df = pd.read_csv("cleaned_all_day_data.csv")
csv_df['data'].iloc[0]

'[[ 1.5942797e-01  9.0304792e-02  1.4607425e-01 -6.4893484e-02\n   4.0591914e-02 -4.3611377e-02]\n [ 1.7132924e-01  7.3759265e-02  1.3082972e-01 -6.0138237e-02\n   5.7521898e-02  1.1591701e-03]\n [ 1.7425483e-01  3.0503687e-02  1.5181470e-01 -6.2486194e-02\n   5.7846423e-02 -1.6662143e-02]\n ...\n [-4.6580547e-01  3.2085699e-01  1.6851538e-01  1.7772797e-02\n  -1.8031813e-03 -4.5005791e-04]\n [-4.5697862e-01  2.9397491e-01  1.7543517e-01  1.3826254e-02\n  -3.5399494e-03  2.9436119e-03]\n [-4.6172807e-01  2.8711575e-01  1.7303370e-01  8.2457634e-03\n  -3.1789739e-03 -6.6284148e-04]]'

In [16]:
# Test with python spark
import os
os.environ["PYTHON"] = "~/.conda/envs/myenv/bin/python3.7" 
os.environ["PYSPARK_PYTHON"] = "~/.conda/envs/myenv/bin/python3.7"
os.environ["PYSPARK_DRIVER_PYTHON"] = "~/.conda/envs/myenv/bin/python3.7"
from pyspark.sql import SparkSession 
import os

spark = SparkSession.builder.master("local")\
.appName("Data")\
.config("spark.some.config.option", "some-value")\
.getOrCreate()

# spark_df= spark.read.options(header=True).csv("cleaned_data.csv")

from pyspark.sql.types import *
mySchema = StructType([ StructField("index", IntegerType(), True)\
                        ,StructField("data", ArrayType(ArrayType(FloatType())), True)\
                            ,StructField("label", IntegerType(), True)])
spark_df = spark.createDataFrame(train_data, mySchema)

ValueError: Length of object (2) does not match with length of fields (3)

In [14]:
# ## using literal_eval to convert string to list
# import pandas as pd
# df = pd.read_csv("test_data.csv")
# from ast import literal_eval
# df['data'] = df['data'].apply(literal_eval)
# df.info()

In [1]:
import os
os.environ["PYTHON"] = "~/.conda/envs/myenv/bin/python3.7" 
os.environ["PYSPARK_PYTHON"] = "~/.conda/envs/myenv/bin/python3.7"
os.environ["PYSPARK_DRIVER_PYTHON"] = "~/.conda/envs/myenv/bin/python3.7"
from pyspark.sql import SparkSession 
import os

spark = SparkSession.builder.master("local")\
.appName("Data")\
.config("spark.some.config.option", "some-value")\
.getOrCreate()

spark_df= spark.read.options(header=True).csv("test_data.csv")
spark_df.printSchema()

from pyspark.sql.types import *
from pyspark.sql.functions import udf
convert = udf(lambda x: literal_eval(x), ArrayType(ArrayType(FloatType())))
new_df = spark_df.withColumn("data", convert(spark_df.data))
!python --version

## My Loader functions

In [2]:
%%writefile data_loader.py
import os
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm
import math
import random
from datetime import datetime
import numba as nb

######################################
# Note: 
#   1. load_PreprocessData is to load all raw data and do smoothing, de-trend processing
#         and then segment series data using lists of indices.
#   2. load_dataset is to return the formatted trainable dataset in pandas dataframe format.
#       It should be run after load_PreProcessData
#   3. loadshmfile is to load .shm file directly and return numpy time series data 
#      There are 6 axis gyroscope, accelerator data in .shm file
#   4. date time information of data is in -events.txt  files, which indicates start,end, eating time period
#      We can use it for more feature extraction
###################################

## Global settings for current dataset
ACC_THRESH = 0.008   # sum(acc) (G) max value for stddev rest
GYRO_THRESH = 0.04 * 180.0/ math.pi
shimmer_global_mean = [-0.012359981,-0.0051663737,0.011612018,
                0.05796114,0.1477952,-0.034395125 ]

shimmer_global_stddev = [0.05756385,0.040893298,0.043825723,
                17.199743,15.311142,21.229317 ]

shimmer_trended_mean = [-0.000002,-0.000002,-0.000000,
                0.058144,0.147621,-0.033260 ]

shimmer_trended_stddev = [0.037592,0.034135,0.032263,
                17.209038,15.321441,21.242532 ]

all_zero_means = [0,0,0,0,0,0]

actigraph_global_means = [ 0.010016, -0.254719, 0.016803, 
                0.430628, 0.097660, 0.359574 ]

actigraph_trended_means = [ -0.000001, 0.000022, -0.000002, 
                0.430628, 0.097660, 0.359574 ]
meanvals = all_zero_means
stdvals = shimmer_trended_stddev



def loadshmfile(File_Name):
    """
    Load shm data file
    Output:
        6-axis gyroscope, accelerator time series data in numpy format
    """
    MB = 1024 * 1024
    RawData = np.fromfile(File_Name, dtype=np.dtype("6f4")) 
    #print(f"File {File_Name} Loaded")
    #print(sys.getsizeof(RawData)/MB)
    # Swap gyroscope axis. Remember python always uses variables with reference.
    # Swap Acceleromter
    Temp = np.copy(RawData[:,5])
    Temp2 = np.copy(RawData[:,3])
    Temp3 = np.copy(RawData[:,4])
    RawData[:,3], RawData[:,4],RawData[:,5] = Temp,Temp2, Temp3
    del Temp
    del Temp2
    del Temp3
    
    return RawData


def smooth(RawData, WINDOW_SIZE = 15,SIG = 10.0 ):
    """
    Smooth 6 axis raw data by convolution using a window
    
    """
    # Create kernel
    # size of window
    r_array = np.linspace(14,0, 15)
    Kernel = np.exp((0-np.square(r_array))/(2 * SIG * SIG))
    deno = sum(Kernel)
    Kernel = Kernel / deno
    del r_array
    del deno
    #Clone (deep copy) the variable, instead of reference. We don't want to change RawData
    Smoothed = np.copy(RawData) 
    r,c = RawData.shape
    for x in range(c):
        # Convolution followed by discarding of extra values after boundary
        Smoothed[:,x] = np.convolve(RawData[:,x], Kernel)[0:len(RawData)]
    # Copy first 15 values from Rawdata to Smoothed. np.convolve doesn't do this.
    Smoothed[:15, :] = RawData[:15,:]
    return Smoothed



def loadEvents(filename):
    """
    loads events data given the .shm filename
    and parse the event.txt file to obtain meal duration
    Input: 
        filename:  <filename>-events.txt name of label file we want to load
    output:
        TotalEvents: amount of event loaded
        EventStart: a list of starting moment of meals
        EventEnd: a list of ending moment of meals
        EventNames: name of meal in string
    """
    # Load the meals file to get any triaged meals.
    SkippedMeals = []
    mealsfile = open("meals-shimmer.txt", "r") 
    for line in mealsfile:
        #print(line)
        data = line.split()
        #print(data[0], data[1], data[13])
        if(int(data[13]) == 0):
            Mdata = [data[0][-9:], data[1], int(data[13])]
            SkippedMeals.append(Mdata)
    
    EventsFileName = filename.replace(".shm","-events.txt")
    
    # Load the meals
    EventNames = []
    EventStart = (np.zeros((100))).astype(int)
    EventEnd = (np.zeros((100))).astype(int)
    TotalEvents = 0
    TimeOffset = 0
    file = open(EventsFileName, "r") 
    #print(filename)
    for lines in file:
        #print(lines)
        words = lines.split()
        if(len(words) == 0): continue # Skip empty lines
        # Convert Start time to offset
        if(words[0] == "START"): # Get Start Time (TimeOffset) from file
            #print(words)
            hours = int(words[2].split(":")[0])
            minutes = int(words[2].split(":")[1])
            seconds = int(words[2].split(":")[2])
            #print("{}h:{}m:{}s".format(hours, minutes,seconds))
            TimeOffset = (hours * 60 * 60) + (minutes * 60) + seconds
            continue
        if(words[0] == "END"):
            #print(words)
            continue
        for x in range(1,3): # Process Events Data
            hours = int(words[x].split(":")[0])
            minutes = int(words[x].split(":")[1])
            seconds = int(words[x].split(":")[2])
            EventTime = (hours * 60 * 60) + (minutes * 60) + seconds
            EventTime = EventTime - TimeOffset
            if(x == 1): EventStart[TotalEvents] = EventTime * 15
            if(x == 2): EventEnd[TotalEvents] = EventTime * 15
        if(TotalEvents>0):
            if(EventStart[TotalEvents]<EventStart[TotalEvents-1]):
                EventStart[TotalEvents] = EventStart[TotalEvents] + (24*60*60*15)
            if(EventEnd[TotalEvents]<EventEnd[TotalEvents-1]):
                EventEnd[TotalEvents] = EventEnd[TotalEvents] + (24*60*60*15)
        #print(TotalEvents)
        
        # Check if meal was triaged out for too much walking or rest
        ename = words[0]
        fname = filename[-9:]
        skipmeal = 0
        #print(fname, ename)
        for skippedmeal in SkippedMeals:
            Pname, EventName, Keep = skippedmeal
            if(Pname == fname and ename == EventName):
                #print(Pname, EventName, Keep, ename, fname, Pname == fname, ename == EventName)
                skipmeal = 1
                break
        
        if(skipmeal == 1): continue
        TotalEvents = TotalEvents + 1
        EventNames.append(ename)
    return TotalEvents, EventStart, EventEnd, EventNames




def load_PreprocessData(winlength, step, meanvals, stdvals,ratio_dataset=1, files_list='batch-unix.txt', 
                 removerest=1, removewalk=0, removebias=1, shx=1, gtperc = 0.5):
    """
    Input:
        winlength: number of data point in a window
        step: number of data point to skip during moving the window forward
        removerest / removewalk: flag to indicate if remove rest/walk period or not
        gtperc:  ground truth percentage:  the ratio of true label in a window to the total window size
                indicating how many data points label = 1 in a window can be regarded as eating d=label
        shx：flag to indicate load shx file or not
        
        ratio_dataset: the ratio of number of days to sample over all days(354 days).
                        It controls the size of dataset. It should be in [0,1]. Otherwise return 0 sample
        
    Output:
        len(df["Filenames"]) : amount of day of dataset
        AllNormalized: normalized, smoothed dataset
        samples_array: list of indices of sample frame in dataset, 
                        format: [(i^th day, start time of window, end time of window),... ]
        labels_array: labels corresponding to each segmentation/window
        
    """
    ### Load data, make samples 

    samples = []
    labels = []
    AllNormalized = []
    AllIndices = []
    totaleatingrest = 0
    totaleatingwalk = 0
    flag = 0
    # load index file of all dataset
    df = pd.read_csv(files_list, names=["Filenames"])
    
    #if ratio is 1, then do nothing
    if ratio_dataset != 1.0:
        if ratio_dataset<1 and ratio_dataset>=0:
            # sample days without replacement
            num_days = int(ratio_dataset* len(df))
            df = df.sample(num_days,replace=False)
            # reorder index
            df.index = [i for i in range(len(df))]
        else:
            df = df.sample(0,replace=False)
        
    
    
    print("Loading Dataset ...")
    # using  tqdm package to visualize process
    for x in tqdm(range(len(df["Filenames"]))):
        fileeatingrest = 0
        fileeatingwalk = 0
        filesamples = []
        filelabels = []
        File_Name = "../Data/" + df["Filenames"][x]
        RawData = loadshmfile(File_Name)
        
        ##################################
        #Data preprocessing, Smoothing here
        #################################            
        # smoothing
        Smoothed = smooth(RawData)
        
        # remove trend of series data
        # Option:  remove acceleration bias or not using a slide window to compute mean
        if(removebias):
            # Remove acceleration bias
            TREND_WINDOW = 150
            mean = []
            for j in range(3):
                dat = pd.Series(Smoothed[:,j]).rolling(window=TREND_WINDOW).mean()
                dat[:TREND_WINDOW-1] = 0
                mean.append(dat)
            mean2 = np.roll(np.asarray(mean).transpose(), -((TREND_WINDOW//2)-1)*3) # Shift to the left to center the values
            # The last value in mean [-75] does not match that of phoneview, but an error in one datum is acceptable
            # The phone view code calculates mean from -Window/2 to <Window/2 instead of including it.
            Smoothed[:,0:3]-=mean2
            del mean2, mean, dat
        
        # Normalization: z-normalization
        Normalized = np.empty_like(Smoothed)
        for i in range(6):
            Normalized[:,i] = (Smoothed[:,i] - meanvals[i]) / stdvals[i]
        # Stick this Normalized data to the Full Array
        AllNormalized.append(np.copy(Normalized))

        
        if(removerest != 0):
        # remove labels for the class of rest 
            std = []
            for j in range(6):
                dat = pd.Series(Smoothed[:,j]).rolling(window=15).std(ddof=0)
                dat[:14] = 0
                std.append(dat)
            # Above doesn't center window. Left Shift all values to the left by 7 datum (6 sensors)
            std2 = np.roll(np.asarray(std).transpose(), -7*6) 
            accstd = np.sum(std2[:,:3], axis=1)
            gyrostd = np.sum(std2[:,-3:], axis=1)
            datrest = (accstd < ACC_THRESH) & (gyrostd < GYRO_THRESH)
            mrest = datrest.copy()

            for i in range(8,len(datrest)-7):
                if(datrest[i]==True):
                    mrest[i-7:i+8] = True
            
            del dat, datrest, gyrostd, accstd, std2, std
        
        if(removewalk!=0):
        # remove labels for the class of walking
        # remove walk period if enabled
            minv = np.zeros((3,1))
            maxv = np.zeros((3,1))
            zerocross = np.zeros((len(Smoothed),1)).astype(int)
            for j in range(3):
                minv[j]=float('inf')
                maxv[j]= -float('inf')

            for t in range(len(Smoothed)-1):
                for j in range(3):
                    if (Smoothed[t][j+3] < minv[j]):
                        minv[j]=Smoothed[t][j+3]
                    if (Smoothed[t][j+3] > maxv[j]):
                        maxv[j]=Smoothed[t][j+3]
                    if ((Smoothed[t][j+3] < 0.0)  and  (Smoothed[t+1][j+3] > 0.0)  and  (minv[j] < -5.0)):
                        zerocross[t]+=(1<<j)
                        minv[j]=float('inf')
                        maxv[j]=-float('inf')
                    if ((Smoothed[t][j+3] > 0.0)  and  (Smoothed[t+1][j+3] < 0.0)  and  (maxv[j] > 5.0)):
                        zerocross[t]+=(1<<(j+3))
                        minv[j]=float('inf')
                        maxv[j]= -float('inf')

            zc = [0 if i==0 else 1 for i in zerocross]
            del minv, maxv, zerocross
        
        del RawData, Smoothed

        
        ###################################
        # Generating labels here
        ###################################
        # Identify things as GT
        [TotalEvents, EventStart, EventEnd, EventNames] = loadEvents(File_Name)
        GT = np.zeros((len(Normalized))).astype(int)
        for i in range(TotalEvents):
            #print(EventStart[i], EventStart[i], type(EventStart[i]))
            GT[EventStart[i]: EventEnd[i]+1] = 1
        
        # Generate labels 
        MaxData = len(Normalized)
        for t in range(0, MaxData, step):
            # x: the x^th day data in dataset
            # t: starting time of eating
            # t+ winlength:  end time of eating
            sample = [x, t, t+winlength]
            label = int((np.sum(GT[t:t+winlength])/winlength)>=gtperc)
            
            #Change labels if the flag of removerest or removewalk is enabled
            if(label and removerest!=0): # Only ignore if in eating
                isrest = int((np.sum(mrest[t:t+winlength])/winlength)>=0.65)
                if(isrest and removerest==1): continue; # Do not consider this sample at all. Comment this if you want to move the sample to non-eating.
                elif(isrest and removerest==2): label = 0;
                else: label = 1    
            if(label and removewalk!=0): # Only ignore if in eating
                iswalk = int((np.sum(zc[t:t+winlength])/winlength)>=0.15)
                if(iswalk and removewalk==1): continue;
                elif(iswalk and removewalk==2): label=0;
                else: label = 1
#                fileeatingwalk+=1
#                continue # Do not append this sample to the dataset
                
            if(t+winlength < MaxData): # Ignore last small window. Not ignoring results in a list rather than a numpy array.
                filesamples.append(sample)
                filelabels.append(label)
                
        #merge two lists
        samples += filesamples
        labels += filelabels
        numsamples = len(filesamples)
        totaleatingwalk += fileeatingwalk

    samples_array = np.asarray(samples)
    labels_array = np.asarray(labels)
    #print("Total {:d} walking in eating\n".format(fileeatingwalk))
    return len(df["Filenames"]), AllNormalized, samples_array, labels_array



def normalizeData(data, meanvals, stdvals):
    """
    Normalize data using given mean values and std variance values
    Input:
         data: time series data set to normalize
         meanvals: global mean value of all days dataset used to smooth per day time series data
         stdvals: global standard variance.
    Output:
        smoothed data
    
    """
    data = []
    for x in range(len(data)):
        Smoothed = data[x]
        Normalized = np.empty_like(Smoothed)
        # Normalize
        for i in range(6):
            Normalized[:,i] = (Smoothed[:,i] - meanvals[i]) / stdvals[i]
        
        # Stick this Normalized data to the Full Array
        AllNormalized.append(np.copy(Normalized))
    
    return AllNormalized




def load_dataset(data, samples_indices, labels_array, shuffle_flag = True, undersampling= True, test_split_ratio =0.3):
    """
    Note: need to run load_PreprocessData to get processed time series data and segmentation data
            before running this function
    Input:
        data: normalized smoothed time series data.

        sample_indices: a list of tuples containing start and end indices of each window in data
                Format:   there are n rows in data, each represents sensor data for one day
                        in each row.
                        There are m x k data, where m = amount of data point along time
                        K = amount of features in training set
        labels_array: label of each segmentation in dataset
    Output:
        pandas data frame with columns "data" and "label"
        used to train model
    """
    from tqdm import tqdm
    import sys
    outfile = sys.stdout
    
    data_indices = samples_indices
    label_indices = labels_array

    # Balance Data here
    #undersample the training dataset
    eatingindices = [i for i, e in enumerate(label_indices) if e >= 0.5]
    noneatingindices = [i for i, e in enumerate(label_indices) if e < 0.5]
    
    #split training set and test set
    eat_len = len(eatingindices)
    noneat_len = len(noneatingindices)
    # Test set indices
    test_noneating = noneatingindices[0: int( noneat_len*test_split_ratio)]
    test_eating  =  eatingindices[0: int(eat_len*test_split_ratio)]  
    test_indices = test_eating + test_noneating
    # Train set indices
    eatingindices = eatingindices[int(eat_len *test_split_ratio):]
    noneatingindices = noneatingindices[int( noneat_len*test_split_ratio): ]
    
    
    
    
    # training set 
    if undersampling:
        underSampledNoneatingIndices = random.sample(noneatingindices,len(eatingindices))
        underSampledBalancedIndices = eatingindices + underSampledNoneatingIndices
        shuffledUnderSampledBalancedIndices = underSampledBalancedIndices.copy()
    else:
        shuffledUnderSampledBalancedIndices = eatingindices+noneatingindices
    
    if shuffle_flag:
        random.shuffle(shuffledUnderSampledBalancedIndices)

    print("Creating balanced training data array", file=outfile, flush=True)
    train_set = pd.DataFrame(columns=['data','label'])
    test_set = pd.DataFrame(columns=['data','label'])

    # use process bar to show the remaining items to handle
    print("Loading training set...")
    for _, i in enumerate(tqdm(shuffledUnderSampledBalancedIndices)):
        f = data_indices[i,0]
        t1, t2 = data_indices[i,1], data_indices[i,2]
        sample,label = data[f][t1:t2], label_indices[i]
        df = pd.DataFrame({'data':[sample], 'label':label})
        train_set = train_set.append(df,ignore_index=True)
    
    print("Loading test set...")
    for _, i in enumerate(tqdm(test_indices)):
        f = data_indices[i,0]
        t1, t2 = data_indices[i,1], data_indices[i,2]
        sample,label = data[f][t1:t2], label_indices[i]
        df = pd.DataFrame({'data':[sample], 'label':label})
        test_set = test_set.append(df,ignore_index=True)
    
    return train_set, test_set


def load_train_test_data(data_path ='batch-unix.txt',
                         ratio_dataset=1, undersampling= True,
                         test_split_ratio = 0.3,
                         winmin = 6, stridesec = 15 ):
#         from datetime import datetime
        ModelNum = 3 
        
        winlength = int(winmin * 60 * 15)
        step = int(stridesec * 15)
#         start_time = datetime.now()
        meanvals, stdvals =  all_zero_means, shimmer_trended_stddev
#     winlength, step, meanvals, stdvals,ratio_dataset=1, files_list='batch-unix.txt', 
#                  removerest=1, removewalk=0, removebias=1, shx=1, gtperc = 0.5
        NumFiles, data, samples_indices, labels_array = load_PreprocessData(winlength,
                                                                                    step,
                                                                                    meanvals, 
                                                                                    stdvals,
                                                                                    ratio_dataset =ratio_dataset,
                                                                                    files_list=data_path, 
                                                                                    removerest=0,
                                                                                    removewalk=0,
                                                                                    removebias=0,
                                                                                    shx=1,
                                                                                     gtperc = 0.5)

        train_set,test_set = load_dataset(data, samples_indices, labels_array,undersampling= undersampling,
                                          test_split_ratio=test_split_ratio)


        return train_set,test_set

Overwriting data_loader.py


In [5]:
df = pd.read_csv("batch-unix.txt", names=["Filenames"])
df = df.sample(10,replace=False)
df.index = [i for i in range(len(df))]
df

,Filenames
0,ShimmerData/P2116/P2116.shm
1,ShimmerData/P2189/P2189.shm
2,ShimmerData/P2410/P2410.shm
3,ShimmerData/P2334/P2334.shm
4,ShimmerData/P2479/P2479.shm
5,ShimmerData/P2289/P2289.shm
6,ShimmerData/P2217/P2217.shm
7,ShimmerData/P2262/P2262.shm
8,ShimmerData/P2302/P2302.shm
9,ShimmerData/P2276/P2276.shm


## Original Load function written by one PhD student

In [8]:
import numpy as np
import pandas as pd
import sys
from tqdm import tqdm
import math

ACC_THRESH = 0.008   # sum(acc) (G) max value for stddev rest
GYRO_THRESH = 0.04 * 180.0/ math.pi

def loadshmfile(File_Name):
    MB = 1024 * 1024
    RawData = np.fromfile(File_Name, dtype=np.dtype("6f4")) 
    #print(f"File {File_Name} Loaded")
    #print(sys.getsizeof(RawData)/MB)
    # Swap gyroscope axis. Remember python always uses variables with reference.
    # Swap Acceleromter
    Temp = np.copy(RawData[:,5])
    Temp2 = np.copy(RawData[:,3])
    Temp3 = np.copy(RawData[:,4])
    RawData[:,3], RawData[:,4],RawData[:,5] = Temp,Temp2, Temp3
    del Temp
    del Temp2
    del Temp3
    
    return RawData


def smooth(RawData, WINDOW_SIZE = 15,SIG = 10.0 ):
    """
    Smooth 6 axis raw data by convolution using a window
    """
    # Create kernel
    # size of window
    r_array = np.linspace(14,0, 15)
    Kernel = np.exp((0-np.square(r_array))/(2 * SIG * SIG))
    deno = sum(Kernel)
    Kernel = Kernel / deno
    del r_array
    del deno
    #Clone (deep copy) the variable, instead of reference. We don't want to change RawData
    Smoothed = np.copy(RawData) 
    r,c = RawData.shape
    for x in range(c):
        # Convolution followed by discarding of extra values after boundary
        Smoothed[:,x] = np.convolve(RawData[:,x], Kernel)[0:len(RawData)]
    # Copy first 15 values from Rawdata to Smoothed. np.convolve doesn't do this.
    Smoothed[:15, :] = RawData[:15,:]
    return Smoothed



def loadEvents(filename):
    """
    loads events data given the .shm filename
    and parse the event.txt file to obtain meal duration
    Input: 
        filename:  <filename>-events.txt name of label file we want to load
    output:
        TotalEvents: amount of event loaded
        EventStart: a list of starting moment of meals
        EventEnd: a list of ending moment of meals
        EventNames: name of meal in string
    """
    # Load the meals file to get any triaged meals.
    SkippedMeals = []
    mealsfile = open("meals-shimmer.txt", "r") 
    for line in mealsfile:
        #print(line)
        data = line.split()
        #print(data[0], data[1], data[13])
        if(int(data[13]) == 0):
            Mdata = [data[0][-9:], data[1], int(data[13])]
            SkippedMeals.append(Mdata)
    
    EventsFileName = filename.replace(".shm","-events.txt")
    
    # Load the meals
    EventNames = []
    EventStart = (np.zeros((100))).astype(int)
    EventEnd = (np.zeros((100))).astype(int)
    TotalEvents = 0
    TimeOffset = 0
    file = open(EventsFileName, "r") 
    #print(filename)
    for lines in file:
        #print(lines)
        words = lines.split()
        if(len(words) == 0): continue # Skip empty lines
        # Convert Start time to offset
        if(words[0] == "START"): # Get Start Time (TimeOffset) from file
            #print(words)
            hours = int(words[2].split(":")[0])
            minutes = int(words[2].split(":")[1])
            seconds = int(words[2].split(":")[2])
            #print("{}h:{}m:{}s".format(hours, minutes,seconds))
            TimeOffset = (hours * 60 * 60) + (minutes * 60) + seconds
            continue
        if(words[0] == "END"):
            #print(words)
            continue
        for x in range(1,3): # Process Events Data
            hours = int(words[x].split(":")[0])
            minutes = int(words[x].split(":")[1])
            seconds = int(words[x].split(":")[2])
            EventTime = (hours * 60 * 60) + (minutes * 60) + seconds
            EventTime = EventTime - TimeOffset
            if(x == 1): EventStart[TotalEvents] = EventTime * 15
            if(x == 2): EventEnd[TotalEvents] = EventTime * 15
        if(TotalEvents>0):
            if(EventStart[TotalEvents]<EventStart[TotalEvents-1]):
                EventStart[TotalEvents] = EventStart[TotalEvents] + (24*60*60*15)
            if(EventEnd[TotalEvents]<EventEnd[TotalEvents-1]):
                EventEnd[TotalEvents] = EventEnd[TotalEvents] + (24*60*60*15)
        #print(TotalEvents)
        
        # Check if meal was triaged out for too much walking or rest
        ename = words[0]
        fname = filename[-9:]
        skipmeal = 0
        #print(fname, ename)
        for skippedmeal in SkippedMeals:
            Pname, EventName, Keep = skippedmeal
            if(Pname == fname and ename == EventName):
                #print(Pname, EventName, Keep, ename, fname, Pname == fname, ename == EventName)
                skipmeal = 1
                break
        
        if(skipmeal == 1): continue
        TotalEvents = TotalEvents + 1
        EventNames.append(ename)
    return TotalEvents, EventStart, EventEnd, EventNames





def loadAllData2(winlength, step, meanvals, stdvals, files_list='batch-unix.txt', 
                 removerest=1, removewalk=0, removebias=1, shx=1, gtperc = 0.5):
    """
    Input:
        winlength: number of data point in a window
        step: number of data point to skip during moving the window forward
        removerest / removewalk: flag to indicate if remove rest/walk period or not
        gtperc:  ground truth percentage:  the ratio of true label in a window to the total window size
                indicating how many data points label = 1 in a window can be regarded as eating d=label
        shx：flag to indicate load shx file or not
    Output:
        len(df["Filenames"]) : amount of day of dataset
        AllNormalized: normalized, smoothed dataset
        samples_array: list of indices of sample frame in dataset, 
                        format: [(i^th day, start time of window, end time of window),... ]
        labels_array: labels corresponding to each segmentation/window
        
    """
    ### Load data, make samples 

    samples = []
    labels = []
    AllNormalized = []
    AllIndices = []
    totaleatingrest = 0
    totaleatingwalk = 0
    flag = 0
    # load index file of all dataset
    df = pd.read_csv(files_list, names=["Filenames"])
    # using  tqdm package to visualize process
    for x in tqdm(range(len(df["Filenames"]))):
        fileeatingrest = 0
        fileeatingwalk = 0
        filesamples = []
        filelabels = []
        File_Name = "../Data/" + df["Filenames"][x]
        RawData = loadshmfile(File_Name)
        
        ##################################
        #Data preprocessing, Smoothing here
        #################################            
        # smoothing
        Smoothed = smooth(RawData)
        
        # remove trend of series data
        # Option:  remove acceleration bias or not using a slide window to compute mean
        if(removebias):
            # Remove acceleration bias
            TREND_WINDOW = 150
            mean = []
            for j in range(3):
                dat = pd.Series(Smoothed[:,j]).rolling(window=TREND_WINDOW).mean()
                dat[:TREND_WINDOW-1] = 0
                mean.append(dat)
            mean2 = np.roll(np.asarray(mean).transpose(), -((TREND_WINDOW//2)-1)*3) # Shift to the left to center the values
            # The last value in mean [-75] does not match that of phoneview, but an error in one datum is acceptable
            # The phone view code calculates mean from -Window/2 to <Window/2 instead of including it.
            Smoothed[:,0:3]-=mean2
            del mean2, mean, dat
        
        # Normalization: z-normalization
        Normalized = np.empty_like(Smoothed)
        for i in range(6):
            Normalized[:,i] = (Smoothed[:,i] - meanvals[i]) / stdvals[i]
        # Stick this Normalized data to the Full Array
        AllNormalized.append(np.copy(Normalized))

        
        if(removerest != 0):
        # remove labels for the class of rest 
            std = []
            for j in range(6):
                dat = pd.Series(Smoothed[:,j]).rolling(window=15).std(ddof=0)
                dat[:14] = 0
                std.append(dat)
            # Above doesn't center window. Left Shift all values to the left by 7 datum (6 sensors)
            std2 = np.roll(np.asarray(std).transpose(), -7*6) 
            accstd = np.sum(std2[:,:3], axis=1)
            gyrostd = np.sum(std2[:,-3:], axis=1)
            datrest = (accstd < ACC_THRESH) & (gyrostd < GYRO_THRESH)
            mrest = datrest.copy()

            for i in range(8,len(datrest)-7):
                if(datrest[i]==True):
                    mrest[i-7:i+8] = True
            
            del dat, datrest, gyrostd, accstd, std2, std
        
        if(removewalk!=0):
        # remove labels for the class of walking
        # remove walk period if enabled
            minv = np.zeros((3,1))
            maxv = np.zeros((3,1))
            zerocross = np.zeros((len(Smoothed),1)).astype(int)
            for j in range(3):
                minv[j]=float('inf')
                maxv[j]= -float('inf')

            for t in range(len(Smoothed)-1):
                for j in range(3):
                    if (Smoothed[t][j+3] < minv[j]):
                        minv[j]=Smoothed[t][j+3]
                    if (Smoothed[t][j+3] > maxv[j]):
                        maxv[j]=Smoothed[t][j+3]
                    if ((Smoothed[t][j+3] < 0.0)  and  (Smoothed[t+1][j+3] > 0.0)  and  (minv[j] < -5.0)):
                        zerocross[t]+=(1<<j)
                        minv[j]=float('inf')
                        maxv[j]=-float('inf')
                    if ((Smoothed[t][j+3] > 0.0)  and  (Smoothed[t+1][j+3] < 0.0)  and  (maxv[j] > 5.0)):
                        zerocross[t]+=(1<<(j+3))
                        minv[j]=float('inf')
                        maxv[j]= -float('inf')

            zc = [0 if i==0 else 1 for i in zerocross]
            del minv, maxv, zerocross
        
        del RawData, Smoothed

        
        ###################################
        # Generating labels here
        ###################################
        # Identify things as GT
        [TotalEvents, EventStart, EventEnd, EventNames] = loadEvents(File_Name)
        GT = np.zeros((len(Normalized))).astype(int)
        for i in range(TotalEvents):
            #print(EventStart[i], EventStart[i], type(EventStart[i]))
            GT[EventStart[i]: EventEnd[i]+1] = 1
        
        # Generate labels 
        MaxData = len(Normalized)
        for t in range(0, MaxData, step):
            # x: the x^th day data in dataset
            # t: starting time of eating
            # t+ winlength:  end time of eating
            sample = [x, t, t+winlength]
            label = int((np.sum(GT[t:t+winlength])/winlength)>=gtperc)
            
            #Change labels if the flag of removerest or removewalk is enabled
            if(label and removerest!=0): # Only ignore if in eating
                isrest = int((np.sum(mrest[t:t+winlength])/winlength)>=0.65)
                if(isrest and removerest==1): continue; # Do not consider this sample at all. Comment this if you want to move the sample to non-eating.
                elif(isrest and removerest==2): label = 0;
                else: label = 1    
            if(label and removewalk!=0): # Only ignore if in eating
                iswalk = int((np.sum(zc[t:t+winlength])/winlength)>=0.15)
                if(iswalk and removewalk==1): continue;
                elif(iswalk and removewalk==2): label=0;
                else: label = 1
#                fileeatingwalk+=1
#                continue # Do not append this sample to the dataset
                
            if(t+winlength < MaxData): # Ignore last small window. Not ignoring results in a list rather than a numpy array.
                filesamples.append(sample)
                filelabels.append(label)
                
        #merge two lists
        samples += filesamples
        labels += filelabels
        numsamples = len(filesamples)
        totaleatingwalk += fileeatingwalk

    samples_array = np.asarray(samples)
    labels_array = np.asarray(labels)
    #print("Total {:d} walking in eating\n".format(fileeatingwalk))
    return len(df["Filenames"]), AllNormalized, samples_array, labels_array



def normalizeData(AllSmoothed, meanvals, stdvals):
    AllNormalized = []
    #for x in tqdm(range(len(AllSmoothed))):
    for x in range(len(AllSmoothed)):
        Smoothed = AllSmoothed[x]
        Normalized = np.empty_like(Smoothed)
        # Normalize
        for i in range(6):
            Normalized[:,i] = (Smoothed[:,i] - meanvals[i]) / stdvals[i]
        
        # Stick this Normalized data to the Full Array
        AllNormalized.append(np.copy(Normalized))
    
    return AllNormalized






## Original  Test example of functions

In [5]:
import sys

### imports
import os
import numpy as np
import pandas as pd
import random
#from sklearn.metrics import classification_report, confusion_matrix
from datetime import datetime


shimmer_global_mean = [-0.012359981,-0.0051663737,0.011612018,
                0.05796114,0.1477952,-0.034395125 ]

shimmer_global_stddev = [0.05756385,0.040893298,0.043825723,
                17.199743,15.311142,21.229317 ]

shimmer_trended_mean = [-0.000002,-0.000002,-0.000000,
                0.058144,0.147621,-0.033260 ]

shimmer_trended_stddev = [0.037592,0.034135,0.032263,
                17.209038,15.321441,21.242532 ]

all_zero_means = [0,0,0,0,0,0]

actigraph_global_means = [ 0.010016, -0.254719, 0.016803, 
                0.430628, 0.097660, 0.359574 ]

actigraph_trended_means = [ -0.000001, 0.000022, -0.000002, 
                0.430628, 0.097660, 0.359574 ]


meanvals = all_zero_means
stdvals = shimmer_trended_stddev

outfile = sys.stdout


ModelNum = 3 #int(sys.argv[1])
EPOCHS = 30 #int(sys.argv[2])
winmin = 6 #int(sys.argv[1])
stridesec = 15 # int(sys.argv[4])

winlength = int(winmin * 60 * 15)
step = int(stridesec * 15)
start_time = datetime.now()

NumFiles, AllNormalized, samples_array, labels_array = loadAllData2(winlength,
                                                                            step,
                                                                            meanvals, stdvals,
                                                                            removerest=0,
                                                                            removewalk=0,
                                                                            removebias=0)
print("Training using every file\n",file=outfile, flush=True)


  0%|          | 0/354 [00:00<?, ?it/s]

(730802, 6)


100%|██████████| 354/354 [02:34<00:00,  2.30it/s]


Training using every file



In [45]:
data = loadshmfile(File_Name= "../Data/ShimmerData/P2001/P2001.shm")

In [46]:
data.shape

(730802, 6)

In [57]:
from sklearn.svm import SVC
clf = SVC(kernel='rbf')
x = train_set['data'].to_list()
ls=[]
num = 800
for i in range(num):
    ls.append(x[i].ravel()) 
# ls[0]
clf.fit(ls, train_set['label'].iloc[:num].tolist())
clf.score(ls,  train_set['label'].iloc[:num].tolist())

0.87375

In [56]:
train_set['label'].iloc[:4].tolist()

[1, 1, 0, 0]

In [58]:
len(ls[0])

32400

In [20]:
import pandas as pd
df = pd.read_csv("cleaned_all_day_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119024 entries, 0 to 119023
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  119024 non-null  int64 
 1   data        119024 non-null  object
 2   label       119024 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 2.7+ MB


In [21]:
df.head()

,Unnamed: 0,data,label
0,0,[[ 1.5942797e-01 9.0304792e-02 1.4607425e-01...,1
1,1,[[ 0.15128006 -0.08849697 -0.1524664 -0.00511...,1
2,2,[[-0.3320187 -0.8915681 -0.8468274 0.13487...,0
3,3,[[-0.52839035 0.26718208 1.2749779 0.18268...,0
4,4,[[ 0.8798676 -0.14580043 1.1407169 0.33000...,1


In [24]:

df["data"].iloc[1:5]

1    [[ 0.15128006 -0.08849697 -0.1524664  -0.00511...
2    [[-0.3320187  -0.8915681  -0.8468274   0.13487...
3    [[-0.52839035  0.26718208  1.2749779   0.18268...
4    [[ 0.8798676  -0.14580043  1.1407169   0.33000...
Name: data, dtype: object